# Exploration des techniques de clustering

Le but de ce tp est de faire face à la problématique: 
<center style="color:red" >  Voici XXX documents -bruts, non étiquetés-... Comment les valoriser? Les exploiter? Les comprendre? Les résumer? </center>

Nous avons vu dans les séances précédentes comment représenter les données textuelles sous forme de sacs de mots:
$$X = 
	\begin{matrix} 
	 & \textbf{t}_j \\
	 & \downarrow \\
	\textbf{d}_i \rightarrow &
	\begin{pmatrix} 
	x_{1,1} & \dots & x_{1,D} \\
	\vdots & \ddots & \vdots \\
	x_{N,1} & \dots & x_{N,D} \\
	\end{pmatrix}
	\end{matrix}
	$$

A partir de cette représentation, les questions qui se posent sont les suivantes:
1. Quel algorithme de clustiering choisir?
    - K-means, LSA, pLSA, LDA
1. Quels résultats attendre?
    - qualité, bruit, exploitabilité immédiate etc...
1. Quelles analyses qualitatives effectuer pour comprendre les groupes?
1. Comment boucler, itérer pour améliorer la qualité du processus?


> <span style="color:magenta" > La tâche est difficile, dans ce TP, on part d'un **jeu de données étiquetées** afin de pouvoir appliquer des métriques quantitatives sur les résultats obtenus. </span>


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import codecs
import re
import os.path
import sklearn

## Chargement des données



In [2]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [3]:
# conversion BoW + tf-idf

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer() # TfidfVectorizer(max_features=500)
vectors = vectorizer.fit_transform(newsgroups_train.data)
print(vectors.shape)

# mesure de la sparsité = 157 mots actif par document sur 130000 !!
print(vectors.nnz / float(vectors.shape[0]))

(11314, 130107)
157.9958458546933


In [8]:
# retrouver les mots  : get feature name donne le mot i 
print([(i,vectorizer.get_feature_names()[i]) \
       for i in np.random.randint(vectors.shape[1], size=10)])

[(49310, 'dualpage'), (95668, 'premices'), (103654, 'rw7h'), (16379, '5w1'), (67521, 'insistence'), (63215, 'hildjj'), (63632, 'hoc_'), (78016, 'm4n'), (20975, '8ekm'), (128919, 'z75pnq')]


In [10]:
# gestion des étiquettes (pour l'évaluation seulemnet en non-supervisé)
Y = newsgroups_train.target # on prend en Y tout les classes des elts
print(Y[:10]) # numérique
print([newsgroups_train.target_names[i] for i in Y[:10]]) # vraie classe, on va prendre le nom correspondant à la clé de Y dans la liste des target _name
print(newsgroups_train.target_names[:10])


[ 7  4  4  1 14 16 13  3  2  4]
['rec.autos', 'comp.sys.mac.hardware', 'comp.sys.mac.hardware', 'comp.graphics', 'sci.space', 'talk.politics.guns', 'sci.med', 'comp.sys.ibm.pc.hardware', 'comp.os.ms-windows.misc', 'comp.sys.mac.hardware']
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball']


# Tests préliminaires

Commençons par le commencement: tout problème non-supervisé (ou presque) doit être analysé en premier lieu avec les $k$-means!


In [11]:
from sklearn.cluster import KMeans
# Algo => risque de prendre du temps si le vocabulaire n'a pas été réduit !!
# Note: on dirait que l'algo transforme les données en dense vector=> catastrophe pour nous !!!
# => limitation du nombre d'itération arbitraire + limitation du vocabulaire

kmeans = KMeans(n_clusters=20, random_state=0, max_iter=10).fit(vectors)

In [12]:
print(kmeans)

KMeans(max_iter=10, n_clusters=20, random_state=0)


In [78]:
# analyse

# recupération des proto:
# cluster = kmeans.cluster_centers_
# cluster_order = np.sort(cluster)
# cluster_order = [end_cluster[-10:] for end_cluster in cluster_order]
# print(cluster_order)
# print(cluster_order)
# mots les plus importants par cluster => TODO
# max_cluster = [max(cluster) for cluster in kmeans.cluster_centers_]
# print([cluster.index(test[0]) for test in cluster_order])
# index_max_cluster = [np.argsort(cluster)[-20:] for cluster in kmeans.cluster_centers_]
# print(index_max_cluster)
lst_max_word = [[vectorizer.get_feature_names()[j] for j in i] for i in [np.argsort(cluster)[-20:] for cluster in kmeans.cluster_centers_]] # vraie classe, on va prendre le nom correspondant à la clé de Y dans la liste des target _name
print(lst_max_word)
# for test in kmeans.cluster_centers_:
#     print(max(test))
# version print / version word cloud

[['0d', 'bxn', 'wm', '2tm', '3t', '1t', 'bhj', 'giz', '75u', '34u', '2di', '0t', 'pl', '145', '1d9', 'a86', 'g9v', 'b8f', 'max', 'ax'], ['for', 'not', 'this', 'have', 'they', 'be', 'as', 'firearms', 'it', 'are', 'is', 'and', 'guns', 'in', 'that', 'you', 'of', 'to', 'gun', 'the'], ['or', 'is', 'sale', 'nntp', 'host', 'com', 'posting', 'thanks', 'organization', 'in', 'lines', 'subject', 'university', 'and', 'from', 'of', 'to', 'for', 'the', 'edu'], ['and', 'in', 'internet', 'to', 'he', 'mule', '30332', 'gt0523e', 'atlanta', 'of', 'go', 'technology', 'edu', 'gtd597a', 'institute', 'hrivnak', 'the', 'georgia', 'prism', 'gatech'], ['not', 'we', 'sgi', 'objective', 'it', 'edu', 'livesey', 'caltech', 'morality', 'and', 'in', 'you', 'god', 'sandvik', 'of', 'keith', 'to', 'is', 'that', 'the'], ['season', 'his', 'for', 'it', 'year', 'ca', 'hockey', 'edu', 'they', 'was', 'is', 'that', 'game', 'team', 'of', 'and', 'in', 'to', 'he', 'the'], ['for', 'government', 'will', 'they', 'this', 'in', 'escro

### Limites

- Limites liées à la description
    - trop de mots
    - trop de mots fréquents qui déroutent l'algorithme
    - ...
- Limites liées à l'algorithme
    - distance euclidienne absurde

Les limites algorithmiques vont être résolues en changeant d'algorithme... Les limites de représentation des données seront résolues par votre capacité en ingénierie.


Algorithmes à tester:
- LSA
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD
- LDA
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

**Note:** pour des tests rapides, il est plus simple de rester dans le cadre de scikit-learn... Néanmoins, dans un milieu industriel, il faudrait exploiter des outils plus efficaces comme ceux présents dans la librairie ```gensim```. Si vous vous sentez à l'aise avec la donnée textuelles, allez directement vers ces outils:
https://radimrehurek.com/gensim/models/ldamodel.html

In [17]:
#   LSA
from sklearn.decomposition import TruncatedSVD
# Algo => risque de prendre du temps si le vocabulaire n'a pas été réduit !!
# Note: on dirait que l'algo transforme les données en dense vector=> catastrophe pour nous !!!
# => limitation du nombre d'itération arbitraire + limitation du vocabulaire

truncatedsvd = TruncatedSVD(n_components=20, n_iter=10, random_state=None).fit(vectors)

In [85]:
# components = truncatedsvd.components_
index_max_cluster = [np.argsort(cluster)[-20:] for cluster in truncatedsvd.components_]
lst_max_word_LSA = [[vectorizer.get_feature_names()[j] for j in i] for i in index_max_cluster] # vraie classe, on va prendre le nom correspondant à la clé de Y dans la liste des target _name
print(lst_max_word_LSA)

/home/gardette/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[['was', 'they', 'as', 'have', 'on', 'are', 'this', 'not', 'be', 'edu', 'for', 'you', 'it', 'that', 'is', 'in', 'and', 'of', 'to', 'the'], ['bible', 'jews', 'armenian', 'by', 'as', 'not', 'who', 'their', 'they', 'jesus', 'people', 'were', 'was', 'his', 'we', 'of', 'that', 'he', 'god', 'the'], ['be', 'morality', 'keith', 'can', 'jesus', 'don', 'people', 'if', 'what', 'are', 'to', 'do', 'we', 'not', 'it', 'your', 'is', 'that', 'god', 'you'], ['this', 'window', 'the', 'can', 'it', 'data', 'government', 'escrow', 'keys', 'scsi', 'be', 'windows', 'use', 'system', 'clipper', 'encryption', 'is', 'chip', 'to', 'key'], ['cwru', 'israeli', 'clipper', 'writes', 'ohio', 'key', 'caltech', 'government', 'keith', 'encryption', 'state', 'article', 'israel', 'gordon', 'banks', 'geb', 'pitt', 'cs', 'of', 'edu'], ['gordon', 'thanks', 'science', 'christ', 'faith', 'cs', 'christians', 'christian', 'card', 'dos', 'drive', 'bible', 'university', 'jesus', 'scsi', 'windows', 'edu', 'is', 'of', 'god'], ['scienc

In [86]:
# LDA
from sklearn.decomposition import LatentDirichletAllocation

latent = LatentDirichletAllocation(n_components=20,max_iter=10,random_state=None).fit(vectors)

In [90]:
np.shape(latent.components_)
index_max_cluster_LDA = [np.argsort(components)[-20:] for components in latent.components_]
lst_max_word_LDA = [[vectorizer.get_feature_names()[j] for j in i] for i in index_max_cluster_LDA] # vraie classe, on va prendre le nom correspondant à la clé de Y dans la liste des target _name
print(lst_max_word_LDA)

/home/gardette/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[['gehrels', 'r5', 'aberystwyth', '124', 'azw', 'haston', 'tempest', 'ghetto', 'aber', 'gazans', 'waii', 'wg2', 'libxmu', 'xmu', 'klinger', 'gec', '1964', 'raider', 'theporch', 'lib'], ['tellabs', 'canseco', 'balltown', 'su', 'ipser', 'blakey', 'cisco', 'alec', 'ohmite', 'chiu', 'scl', 'skybridge', 'ricardo', 'macadam', 'welty', 'jcj', 'belton', 'ozonehole', 'johnh', 'mpce'], ['mr2', 'altima', 'lick', 'vpnet', 'norway', 'bj', 'joslin', 'sigurdsson', 'steinn', 'dominance', 'cain', 'parsli', 'sabbath', 'steinly', 'topaz', 'bears', 'pooh', 'thomasp', 'uio', 'halat'], ['with', 'from', 'not', 'are', 'have', 'be', 'com', 'on', 'this', 'for', 'you', 'edu', 'it', 'that', 'is', 'in', 'and', 'of', 'to', 'the'], ['ma90jjw', 'numlock', '_______________________________', 'vcd', 'klossner', '581', 'ailin', 'adjective', 'wv', 'koppenhoefer', 'cramm', 'kkopp', 'nevada', 'stramer', 'supergas', 'frip', 'naftaly', 'downs', 'dazixco', 'nstramer'], ['notebooks', 'waking', 'ualberta', 'cfs', 'sphughes', 'sh

In [99]:
import pyLDAvis
pyLDAvis.enable_notebook()
import json


newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
docs_raw = newsgroups.data
print(len(docs_raw))


# prepare_viz = pyLDAvis(latent.components_)
# latent_json = json.load(latent)
# pyLDAvis.display(latent)

11314


In [105]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(11314, 9144)


In [108]:
import pyLDAvis.sklearn
pyLDAvis.sklearn.prepare(latent, vectors, vectorizer, mds='mmds')

/home/gardette/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/gardette/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/gardette/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/gardette/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.308228  0.284840       1        1  72.167069
6      0.022534 -0.090680       2        1   3.058936
12    -0.073877 -0.053421       3        1   2.625435
19    -0.032397  0.042074       4        1   2.013814
8      0.021261 -0.034705       5        1   1.696641
18     0.009191 -0.026355       6        1   1.423072
7     -0.007749  0.017428       7        1   1.418986
17    -0.027316 -0.037973       8        1   1.402285
11    -0.047260  0.002215       9        1   1.399680
10     0.003320  0.005012      10        1   1.326154
16     0.014343 -0.004818      11        1   1.322416
1     -0.017728  0.005593      12        1   1.298752
9     -0.044319 -0.014075      13        1   1.290551
2     -0.031793  0.007776      14        1   1.261935
0     -0.013078 -0.030712      15        1   1.127869
4     -0.022226 -0.007463      16        1   1.088953
14    -0.017831 -0.020672      17        1   1.045894
13    -0.028812 -0.016562      18        1   1.033028
5     -0.009725 -0.016152      19        1   1.015381
15    -0.004765 -0.011351      20        1   0.983148, topic_info=         Term        Freq       Total Category  logprob  loglift
114455    the  974.000000  974.000000  Default  30.0000  30.0000
115475     to  542.000000  542.000000  Default  29.0000  29.0000
89362      of  494.000000  494.000000  Default  28.0000  28.0000
28146     and  416.000000  416.000000  Default  27.0000  27.0000
66608      in  381.000000  381.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
82227    ming    0.133403    0.497701  Topic20  -8.9195   3.3055
68604    isis    0.164765    0.767412  Topic20  -8.7084   3.0837
34786   brake    0.141655    0.925657  Topic20  -8.8595   2.7451
111259     su    0.141923    1.611672  Topic20  -8.8576   2.1924
73843    kurt    0.123984    2.525883  Topic20  -8.9928   1.6080

[847 rows x 6 columns], token_table=        Topic      Freq     Term
term                            
875        13  1.339772     0358
6216        2  0.639405     1920
11475      13  1.316607    30602
12655       3  1.074994     3com
15266      13  1.157684      542
...       ...       ...      ...
128402      1  1.000991      you
128529      1  0.909075      ysu
128529     11  0.181815      ysu
129427      2  0.740834  zionism
129935      2  0.893355     zuma

[280 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 13, 20, 9, 19, 8, 18, 12, 11, 17, 2, 10, 3, 1, 5, 15, 14, 6, 16])

## Evaluation des performances

Les performances sont très dures à évaluer en clustering... Ce qui explique que cette évaluation est souvent, au moins partiellement, qualitative (=étudiée à l'oeil, sur des exemples ou des paramètres). 
Afin d'éviter de faire n'importe quoi, il faut aussi réfléchir à des métriques quantitatives.

### Qualitatif

Analyser le vocabulaire des différents clusters
1. En terme de mots les plus fréquents, les plus probables ou de dimensions des vecteurs propres les plus fortes.
1. En terme de mots discriminants
    - construction de critère de contraste (type odd's ratio) entre la présence dans une classe et présence dans les autres classe
1. Affichage 
    - des 10 ou 20 mots critiques de chaque classe ```print```
    - sous la forme de word cloud
    - affichage interactif avancé: http://www.kennyshirley.com/LDAvis/
        - pour une version intégrable dans un notebook: https://github.com/bmabey/pyLDAvis
        - merci de l'utiliser **après avoir compris le principe de réduction de la dimensionalité pour les clusters**
    
### Quantitatif

Pour donner des chiffres, il faut des étiquettes. C'est rarement le cas sur des jeux de données industriels... Mais c'est bon dans un cadre académique comme 20 newsgroups!

**Problème:** Comme nos algorithmes sont non supervisés, les sorties (bien que catégorielles) ne sont pas alignées avec l'encodage des étiquettes du jeu de données. Il faut trouver des astuces.

1. Etude basique sur la taille des clusters
    - est ce qu'une classe n'a pas tout attrapé?
1. Pureté = extraction de la classe majoritaire dans un cluster + calcul de la pureté du cluster
    - 1 score par cluster par défaut
    - agrégation par somme pondérée sur la taille des clusters
1. Indice de Rand  https://fr.wikipedia.org/wiki/Indice_de_Rand
1. Métrique adaptée à une hypothèse spécifique


## Vers une version plus évoluée des algorithmes

1. Si l'un des clusters attiré toutes les données: êtes-vous capable de supprimer ce cluster et de simplement répartir les échantillons dans les autres catégories?

1. Si vous avez une idée vague des thématiques que vous souhaitez voir isolées... 
    - trouver 10 mots dans chaque catégories
    - biaiser l'initialisation pour attirer ces classes

1. Si vous mettez un utilisateur dans la boucle
    - passer en mode supervisé multiclasse et exploiter les feedbacks de l'utilisateur pour forcer le passage d'un document dans la classe d'à coté 
        - Naive Bayes, SVM ou autre...
    - réfléchir à une approche active qui sélectionne les documents les plus intéressants à montrer à l'utilisateur pour lui demander un étiquetage.